# Mapping CAIT

The objective of thsi notebook is to map the CAIT dataset to the Opengeoscale model. 

CAIT data can be retrieved from several ways on the ClimateWatch tool online. 
The most robust method seems to click on "Download Bulk Data", and select "GHG Emissions". The dataset is is named CW_CAIT_GHG_Emissions.xlsx


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import math

pd.set_option("max_rows", 200)

<h2> Read the file </h2>

In [2]:
# Read datafile 
df_ghg = pd.read_excel("../../../data/ghg-emissions/wri/CW_CAIT_GHG_Emissions.xlsx", header=0)

#drop useless header 
df_ghg.drop("Source", axis=1, inplace=True)

# Set the dataframe structure and create a multiindex structure in country-gas-sector 
df_ghg.set_index(["Country", "Gas", "Sector"], inplace=True)


In [3]:
df_ghg[0:100]

1990       1991  \
Country Gas     Sector                                                
AFG     All GHG Total excluding LUCF           15.182850  15.102010   
                Total including LUCF           12.794040  12.713210   
                Energy                          5.829497   5.334624   
                Industrial Processes            0.051879   0.054500   
                Agriculture                     8.072853   8.396465   
                Waste                           1.228617   1.316422   
                Land-Use Change and Forestry   -2.388800  -2.388800   
                Bunker Fuels                    0.019000   0.019000   
                Electricity/Heat                0.268000   0.265000   
                Manufacturing/Construction      0.571000   0.532000   
                Transportation                  1.672000   1.550000   
                Building                        0.081000   0.071000   
                Other Fuel Combustion           2.628771   2.401968   
                Fugitive Emissions              0.609725   0.515656   
        CO2     Total excluding LUCF            2.964180   2.735423   
                Total including LUCF            0.575377   0.346620   
                Energy                          2.920240   2.691533   
                Industrial Processes            0.044940   0.044890   
                Land-Use Change and Forestry   -2.388800  -2.388800   
                Bunker Fuels                    0.019000   0.019000   
                Electricity/Heat                0.268000   0.265000   
                Manufacturing/Construction      0.571000   0.532000   
                Transportation                  1.672000   1.550000   
                Building                        0.081000   0.071000   
                Other Fuel Combustion           0.000000   0.000000   
                Fugitive Emissions              0.328240   0.273533   
        CH4     Total excluding LUCF            8.965122   9.066115   
                Total including LUCF            8.965122   9.066115   
                Energy                          2.508165   2.286496   
                Industrial Processes            0.000000   0.000000   
                Agriculture                     5.364480   5.606798   
                Waste                           1.092478   1.172821   
                Land-Use Change and Forestry    0.000000   0.000000   
                Other Fuel Combustion           2.226680   2.044374   
                Fugitive Emissions              0.281485   0.242123   
        N2O     Total excluding LUCF            3.246604   3.290863   
                Total including LUCF            3.246604   3.290863   
                Energy                          0.402092   0.357595   
                Industrial Processes            0.000000   0.000000   
                Agriculture                     2.708373   2.789667   
                Waste                           0.136139   0.143601   
                Land-Use Change and Forestry    0.000000   0.000000   
                Other Fuel Combustion           0.402092   0.357595   
                Fugitive Emissions              0.000000   0.000000   
        F-Gas   Total excluding LUCF            0.006939   0.009610   
                Total including LUCF            0.006939   0.009610   
                Industrial Processes            0.006939   0.009610   
AGO     All GHG Total excluding LUCF           42.175630  42.842020   
                Total including LUCF           65.179160  65.845540   
                Energy                         11.399100  11.962490   
                Industrial Processes            0.112110   0.113991   
                Agriculture                    30.138430  30.221310   
                Waste                           0.525987   0.544229   
                Land-Use Change and Forestry   23.003520  23.003520   
                Bunker Fuels                    1.056000   0.911000   
     

# Mapping GHG Emissions 

### Definition of the dataframe sectors

In [4]:
# Definition of the sub-sectors list
# and heir mapped conunterpart
mapped_sectors={'Total excluding LUCF': 'total_excluding_LUCF',
                'Total including LUCF': 'total_including_LUCF',
                'Energy ': 'total_energy', 
                'Industrial Processes ': 'industrial_processes',
                'Agriculture ':'agriculture',
                'Waste ':'waste',
                'Land-Use Change and Forestry ': 'lucf', 
                'Bunker Fuels ':'bunker_fuels',
                'Electricity/Heat ':'electricity_heat', 
                'Manufacturing/Construction ':'manufacturing_construction',
                'Transportation ':'transportation', 
                'Building':'building',
                'Other Fuel Combustion ': 'other_fuel_combustion',
                'Fugitive Emissions ': 'fugitive_emissions'
               }

#The present gases 
#Their mapped counterparts
mapped_gases={'All GHG': 'kyoto_gases',
            'CO2': 'co2',
            'CH4': 'ch4',
            'N2O': 'n2o',
            'F-Gas':'f-gas',
           }



In [5]:
# Define the name for the output mapped datafile.
path="../../../data/ghg-emissions/wri/mapped/"
data_name="wri-cait"
namefile = path + "mapped_data_"+data_name+".json"

In [6]:
def MapDataSet(df, namefile):
    '''
    Function meant to map a dataframe provided with country-gas-sector, and years in columns
    '''

    # Create the output file
    file = open(namefile, "w")
    
    #list of years
    years = df.columns
    
    #list of countries
    countries = df.index.get_level_values(0).unique()[0:2]

    
    # Loop over countries
    for country in countries:
        
        print(country)
        #list of gases
        gases = df.loc[country,:].index.get_level_values(0).unique()
                 
        #Loop over gases 
        for gas in gases:
                        
            #list of sectors
            sectors = df.loc[(country,gas),:].index.get_level_values(0).unique()
            
            #Loop over sectors
            for sector in sectors:
                    
                for year in years:   
                                                            
                    #Get the emission value from dataset
                    emission_value = df.loc[(country,gas,sector),year]
                    
                    #We have to create a json block for each emission value. The method is borrowed from UNFCCC
                    if not math.isnan(emission_value):

                        # Create dictionary that will contain the final json object
                        dict_mapped_entry={}

                        #Entries associated with data_source
                        dict_data_source={}
                        
                        # Sub-dictionary data_source properties
                        dict_data_source_properties={}
                        dict_data_source_properties["description"]=None
                        dict_data_source_properties["provider"]="CAIT"
                        
                        dict_data_source["name"]=data_name    
                        dict_data_source["link"]="https://www.climatewatchdata.org/"
                        dict_data_source["properties"]=dict_data_source_properties

                        # Entries associated with geo_component
                        dict_geo_component={}

                        # Sub-dictionary geo-component ID
                        dict_geo_component_id={}
                        dict_geo_component_id["id"]=country
                        dict_geo_component_id["type"]="alpha3"
                        
                        # Sub-dictionary geo-component properties
                        dict_geo_component_properties={}
                        dict_geo_component_properties["data_source_code"]=None

                        dict_geo_component["scale"]="country"
                        dict_geo_component["name"]=None      
                        dict_geo_component["identifier"]=dict_geo_component_id
                        dict_geo_component["properties"]=dict_geo_component_properties

                        #Entries associated with emissions
                        dict_emission={}

                        # Sub-dictionnary on unit
                        dict_emission_unit={}
                        dict_emission_unit["unit_used"]="Mt co2eq"

                        # Sub-dictionary on sector
                        dict_emission_sector={}
                        dict_emission_sector["sector_origin_name"]=sector
                        dict_emission_sector["sector_mapped_name"]=mapped_sectors[sector]

                        dict_emission["gas"]=mapped_gases[gas]
                        dict_emission["value"]=emission_value
                        dict_emission["unit"]=dict_emission_unit
                        dict_emission["sector"]=dict_emission_sector         

                        #Combine everything together
                        dict_mapped_entry["data_source"]=dict_data_source
                        dict_mapped_entry["geo_component"]=dict_geo_component 
                        dict_mapped_entry["date"]=str(year)+"-01-01"
                        dict_mapped_entry["emission"]=dict_emission
                    
                        # Write the json object to a file and add a line break (every line is a json object)
                        json.dump(dict_mapped_entry, file)
                        file.write("\n")                        

    file.close()
                


In [7]:
#map the data
MapDataSet(df_ghg, namefile)

AFG


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\indexing.py:889: PerformanceWarning: indexing past lexsort depth may impact performance.
  return self._getitem_tuple(key)


AGO
